<a href="https://colab.research.google.com/github/AngelChv/wordle/blob/master/wordle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wordle

In [ ]:
!pip install rich
!pip install wordfreq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
import json
import os
from json import JSONDecodeError
import random
import re
import requests
from regex import Pattern
from requests import Response, RequestException
from rich.console import Console
import re
from typing import Callable, Pattern
from rich.table import Table
import wordfreq
from typing import Callable, Pattern

In [ ]:
console = Console()

## Word

In [ ]:
class Word:
    """
    Almacena el estado de cada uno de los caracteres de la palabra.
    Si el carácter coincide en la misma posición que el de la palabra a adivinar, se mostrará en verde.
    Si no coincide la posición, pero sí aparece en la palabra, se mostrará en amarillo.
    Pero si no aparece, se mostrará en gris.
    """
    def __init__(self, word: str):
        self.characters: list[str] = list(word)

    def check(self, word: str) -> bool:
        win = True
        for i, c in enumerate(self.characters):
            positions = [index for index, char in enumerate(word) if c == char]
            if positions:
                if i in positions:
                    self.characters[i] = f"[green]{c}[/]"
                else:
                    win = False
                    self.characters[i] = f"[yellow]{c}[/]"
            else:
                win = False
                self.characters[i] = f"[dim]{c}[/]"
        return win


    def get_characters(self) -> list[str]:
        return self.characters

## Resource Manger:

In [ ]:
def get_resource(name: str) -> list[str]:
    """
    Función para obtener una lista de palabras de un recurso local.
    :return: Lista de palabras.
    """
    try:
        with open("resources/" + name, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"El archivo {name} no fue encontrado.")
        return []
    except JSONDecodeError:
        print(f"Error al decodificar el archivo JSON: {name}.")
        return []
    except OSError as e:
        print(f"Error de entrada/salida: {e}")
        return []


def set_words(words: list[str], name: str) -> None:
    """
    Almacena una lista de palabras en formato json.
    :param name: Nombre del recurso.
    :param words: Lista de palabras.
    :return: :class:`None <None>`
    """
    if not os.path.exists('resources'):
        os.mkdir("resources")

    try:
        with open('resources/' + name, 'wt', encoding='utf-8') as f:
            json.dump(words, f)
    except OSError as e:
        print(f"No se han podido guardar las palabras: {e}")


## Data Muse

In [ ]:
def get_rand_word_datamuse() -> str:
    """
    Genera una palabra aleatoria en función a una lista obtenida de la api datamuse mediante una petición o de un
    archivo local.
    :return: Palabra aleatoria.
    """
    words: list[str] = get_resource("datamuse.json")
    rand_word: str
    if words:
        console.print("Palabras cargadas del fichero local (datamuse.json).")
        rand_word = random.choice(words)
    else:
        console.log("Descárgando...")
        response: Response = get_request()
        # Compruebo el estado de la petición (200 es correcta).
        if response.status_code == 200:
            words = [w['word'] for w in response.json()]
            if words:  # compruebo si la respuesta no es nula.
                filtered_words = filter_words(words)
                set_words(filtered_words, "datamuse.json")
                rand_word = random.choice(filtered_words)
            else:
                console.print("Error no se han descargado palabras.", style="red")
                rand_word = f"Error, no se han descargado palabras."
        else:
            rand_word = f"Error al conectarse a la API: {response.status_code}"
    return rand_word


def get_request() -> Response:
    """
    Realiza una petición a la API datamuse para obtener una serie de palabras con 5 letras en español.
    :return: :class:`Response <Response>` que almacena el código de estado y el contenido de la petición.
    """
    # Utilizo la api datamuse para obtener palabras.
    url: str = 'https://api.datamuse.com/words'
    # parámetros que definen la cantidad de palabras y la condición.
    params: dict[str, str | int] = {
        'sp': '?????',
        'max': 1000,
        'v': 'es'
    }
    try:
        # realizo la petición:
        return requests.get(url, params=params)
    except RequestException as e:
        console.print(f"Error en la petición a la API: {e}", style="red")
        # Devolver una respuesta vacía para que se detecte como error.
        return Response()


def filter_words(words: list[str]) -> list[str]:
    word_length: int = 5
    regex: Pattern = re.compile(fr"^[a-zA-ZáéíóúÁÉÍÓÚñÑ]{{{word_length}}}$")
    return [word for word in words if regex.match(word)]


## Word Freq

In [ ]:
def get_rand_word_freq() -> str:
    """
    Genera una palabra aleatoria en función a una lista obtenida de la librería wordfreq.
    :return: Palabra aleatoria.
    """
    words: list[str] = get_resource("wordfreq.json")
    rand_word: str
    if words:
        print("Palabras cargadas del fichero local (wordfreq.json).")
        rand_word = random.choice(words)
    else:
        print("Recurso de palabras wordfreq no encontrado. Generando...")
        words = wordfreq.top_n_list('es', 1000) # Obtener las 1000 palabras más comunes.
        filtered_words: list[str] = [p for p in words if len(p) == 5]
        set_words(filtered_words, "wordfreq.json")
        rand_word = random.choice(filtered_words)

    return rand_word

## Main

In [ ]:
def main():
    """
    todo guardar puntuaciones.
    Author: <Ángel Chicote>
    Consejos: ejecutar si no vés los colores la terminal de tu ide, ejecutaló en la terminal directamente.
    Instalaciones:
    pip install rich
    """
    console.rule("Bienvenido al [bold][red]W[/][green]o[/][yellow]r[/][blue]d[/][magenta]l[/][cyan]e[/][/] para terminal!")

    #Pedir al usuario de qué forma quiere obtener las palabras:
    print("Este programa necesita cargar una lista de palabras, para ello existen dos opciones:")
    print("- Usar data muse (una api que proporciona palabras en inglés y en español) no requiere de instalación. "
          "programa funciona en español, pero la api muchas veces confunde palabras en ingles y las introduce en la lista.")
    print("- Wordfreq, es una librería de python que proporciona una serie de palabras comunes, necesita de instalación, "
          "pero proporcióna palabras de uso frecuente más fáciles de adivinar.")
    op = request_int(
        "[blue]1. DataMuse.\n2. Wordfreq.\nElige: ",
        lambda o: True if re.match(f"^[12]$", o) else False
    )

    # En función de la opción elegida, se carga un módulo u otro.
    try:
        match op:
            case 1: hidden_word: str = get_rand_word_datamuse()
            case 2: hidden_word: str = get_rand_word_freq()
            case _: raise RuntimeError("No se ha elegído una opción válida para el módulo de carga de palabras.")

        # Crear una tabla
        table = Table(title="[bold cyan]Wordle[/]", show_header=False, style="magenta")
        # Longitud de la palabra.
        word_length: int = 5
        # Patrón que deben cumplir las palabras introducidas.
        regex: Pattern = re.compile(fr"^[a-zA-ZáéíóúÁÉÍÓÚñÑ]{{{word_length}}}$")
        win: bool = False # almacena si el jugador ha ganado.
        turn: int = 1 # turno actual
        attempts: int = 6 # máximo de rondas.

        console.print("Tienes [bold]5 turnos[/] para adivinar la palabra oculta.")
        console.print("En cada intento deberás proporcionar una palabra de [bold]5 letras[/].")
        console.print("Si una letra está en la misma posición que la palabra oculta, aparecerá en [green]verde[/].")
        console.print("Si una letra está en la palabra, pero no en la misma posición, aparecerá en [yellow]amarillo[/].")
        console.print("Si una letra no está en toda la palabra, aparecerá en [dim]gris[/].")

        # El bucle de juego continúa si no se han acabado las rondas ni el jugador ha ganado.
        while turn <= attempts and not win:
            # Crear objeto Word con la palabra pedida por teclado, la cual se validará con el regex.
            player_word: Word = Word(request_str(f"{turn}: ", lambda word: True if regex.match(word) else False))
            #Comprobar la palabra y establecer los colores de cada carácter.
            win = player_word.check(hidden_word)
            # Añadir filas a la tabla, cada carácter es una columna, por lo tanto, utilizo '*' para separar los elementos
            # de la lista de carácteres en los diferentes argumentos de la función.
            table.add_row(*player_word.characters)
            # Imprimir la tabla
            console.print(table)
            # Incrementar turno
            turn += 1

        if win: # Victoria.
            console.print("[green]Has ganado![/]")
        else: # Derrota.
            console.print(f"[red]Has perdido, la palabra era: {hidden_word}[/]")

        if console.input("[underline]Quieres seguir jugando? (s/n): ").strip().lower() == 's': main()
    except ModuleNotFoundError as mnfe:
        console.print("No se ha podido encontrar el módulo que se intenta cargar: ", mnfe, style="red")
    except RuntimeError as rune:
        console.print(rune, style="red")




def request_str(message: str, validator: Callable[[str], bool]) -> str:
    try:
        string: str = console.input(message).strip().lower()
        if string and validator(string):
            return string
        else:
            console.print("La palabra no es válida.", style="yellow")
            return request_str(message, validator)
    except UnicodeDecodeError as ude:
        console.print("Error: ", ude, style="red")
        return request_str(message, validator)


def request_int(message: str, validator: Callable[[str], bool]) -> int:
    try:
        num: str = console.input(message).strip()
        if num and validator(num):
            return int(num)
        else:
            console.print("El número no es válido.", style="yellow")
            return request_int(message, validator)
    except (UnicodeDecodeError, ValueError, TypeError) as e:
        console.print("Error: ", e, style="red")
        return request_int(message, validator)


if __name__ == '__main__':
    main()

─────────────────────────────────────── Bienvenido al Wordle para terminal! ───────────────────────────────────────

Este programa necesita cargar una lista de palabras, para ello existen dos opciones:
- Usar data muse (una api que proporciona palabras en inglés y en español) no requiere de instalación. programa funciona en español, pero la api muchas veces confunde palabras en ingles y las introduce en la lista.
- Wordfreq, es una librería de python que proporciona una serie de palabras comunes, necesita de instalación, pero proporcióna palabras de uso frecuente más fáciles de adivinar.


1. DataMuse.
2. Wordfreq.
Elige: 

2
Palabras cargadas del fichero local (wordfreq.json).


Tienes 5 turnos para adivinar la palabra oculta.

...

serio


       Wordle        
┌───┬───┬───┬───┬───┐
│ a │ e │ i │ o │ u │
│ b │ c │ d │ f │ g │
│ j │ l │ m │ n │ p │
│ r │ s │ t │ v │ z │
│ s │ e │ r │ i │ o │
└───┴───┴───┴───┴───┘

Has ganado!

Quieres seguir jugando? (s/n): 

n
